In [6]:
import sys
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from os.path import join 
import json

import nibabel as nib
from nilearn.plotting import plot_stat_map, plot_anat, plot_img, show

from nistats.first_level_model import FirstLevelModel
from nistats.reporting import plot_design_matrix, make_glm_report
from nistats.thresholding import map_threshold
from nistats.design_matrix import make_first_level_design_matrix

path_root = os.environ.get('DECIDENET_PATH')
path_code = os.path.join(path_root, 'code')
if path_code not in sys.path:
    sys.path.append(path_code)
from dn_utils.behavioral_models import load_behavioral_data                    
from dn_utils.glm_utils import Regressor, my_make_first_level_design_matrix

%matplotlib inline

In [14]:
path_sourcedata = join(path_root, 'data/main_fmri_study/sourcedata')
path_derivatives = join(path_root, 'data/main_fmri_study/derivatives') 

path_data_paths = join(path_derivatives, 'data_paths')
path_beh = join(path_sourcedata, 'behavioral')

In [20]:
# Load behavioral data
beh, meta = load_behavioral_data(path=path_beh)

# Load path to data files
with open(join(path_data_paths, 'fmri_filenames.json'), 'r') as f:
    fmri_files = json.loads(f.read())
with open(join(path_data_paths, 'conf_filenames.json'), 'r') as f:
    conf_files = json.loads(f.read())
with open(join(path_data_paths, 'mask_filenames.json'), 'r') as f:
    mask_files = json.loads(f.read())    

Shape of beh array: (32, 2, 110, 23)
Conditions [(0, 'rew'), (1, 'pun')]
Columns: [(0, 'block'), (1, 'block_bci'), (2, 'side'), (3, 'side_bci'), (4, 'magn_left'), (5, 'magn_right'), (6, 'response'), (7, 'rt'), (8, 'won_bool'), (9, 'won_magn'), (10, 'acc_after_trial'), (11, 'onset_iti'), (12, 'onset_iti_plan'), (13, 'onset_iti_glob'), (14, 'onset_dec'), (15, 'onset_dec_plan'), (16, 'onset_dec_glob'), (17, 'onset_isi'), (18, 'onset_isi_plan'), (19, 'onset_isi_glob'), (20, 'onset_out'), (21, 'onset_out_plan'), (22, 'onset_out_glob')]


In [24]:
# Times of image acquisition in seconds
n_scans, t_r = 730, 2
frame_times = np.arange(n_scans) * t_r

# GLM parameters
glm_kwargs = {
    't_r': 2,
    'hrf_model': 'spm',
    'drift_model': 'cosine',
    'noise_model': 'ar1',
    'high_pass': 0.0078125, 
    'standardize': True,
    'smoothing_fwhm': 6,
}

# Later loop over these
sub, sub_idx = 'm02', 0
con, con_idx = 'prlrew', 0
trial = 0

In [29]:
# Load subject data
fmri_img = nib.load(fmri_files[con][sub_idx])

confounds = pd.read_csv(conf_files[con][sub_idx], sep='\t')
confounds = confounds.filter(regex='trans|rot')
confounds.index = frame_times # Standard time representation (in seconds)



In [31]:
con_label

'rew'